# Увод и опис проблема

У овој вежби ћемо научити како се већ истрениран модел на неком сету података може адаптирати на други сет података.

**У задацима попунити само део кода означен са 3 тарабе (###).**

Имплементирти функцију **image_loader** која учитава слику са задате локације **path**, трансформише копију слике из BGR (0-255) формата који користи OpenCV у RGB (0.0 - 1.0),а затим враћа **Tensor** слике.

In [1]:
import torch
import cv2
import typing
from testbench import Testbench

ModuleNotFoundError: No module named 'testbench'

In [ ]:
def image_loader(path: str) -> torch.Tensor:
    image = (cv2.imread(path).astype("float32") / 255.0)[:, :, ::-1].copy()
    return torch.from_numpy(image.transpose(2, 0, 1))

In [ ]:
'''Test'''
Testbench(image_loader)

Имплементирати callable класе **RandomGamma** и **ClipImage**.
<ul>
    <li>RandomGamma - Ствара се са аргументом <em>random_gamma_delta</em>. Када се позове примењује гама филтер са вредношћу која има униформну расподелу у интервалу [1 - <em>random_gamma_delta</em> , 1 + <em>random_gamma_delta</em> ]</li>
    <li>ClipImage - Ствара се без аргумената. Када се позове клипује све вредности у интервал [0, 1]</li>
</ul>

In [ ]:
import numpy as np
class ClipImage():
    def __call__(self, image):
        return np.clip(image, 0.0, 1.0)

    def __repr__(self):
        return 'ClipImage()'

class RandomGamma():
    def __init__(self, random_gamma_delta):
        self.gamma_range = 1.0 - random_gamma_delta, 1.0 + random_gamma_delta

    def __call__(self, image):
        return np.power(image, np.random.uniform(*self.gamma_range))

    def __repr__(self):
        return 'RandomGamma('+str(self.gamma_range)+')'

In [ ]:
def getRandomGamma(randaom_gamma_delta : float) -> torch.Tensor:
    return RandomGamma(randaom_gamma_delta)

def getClipImage() -> torch.Tensor:
    return ClipImage()

In [ ]:
'''Test'''
Testbench(getRandomGamma)
Testbench(getClipImage)

Имплементирати функцију **composed_transforms** која враћа композицију следећих трансформација и филтера:

<ul>
    <li>RandomGamma филтер са <em>random_gamma_delta</em> = 0.3,</li>
    <li>ClipImage, </li>
    <li>Трансформација у PLI слику, </li>
    <li>Промена величине слике tako da W=H=224, </li>
    <li>Произвољни оризонтални флип, </li>
    <li>Произвољни вертикални флип, </li>
    <li>Трансфформација натраг у тенсор и </li>
    <li>Нормализацију сваког канала: R(mean=0.485, std=0.229), G(mean=0.456, std=0.224), B(mean=0.406, std=0.225) </li>
</ul>

In [ ]:
import torchvision

In [ ]:
def compose_transforms():
    return torchvision.transforms.Compose([
    ###
])


In [ ]:
'''Test'''
Testbench(compose_transforms)

Даље, учитавамо сет података.

In [2]:
dataset_path = 'datasets/2750'
dataset = torchvision.datasets.DatasetFolder(root=dataset_path, loader=image_loader, transform=compose_transforms(), extensions="jpg")

NameError: name 'torchvision' is not defined

 Напиши функцију **split_data** која као аргументе прима листу са односом броја података за тренинг, тестирање и евалуацију, и укупан број података, а враћа број података за тренинг, тестирање и евалуацију.

In [ ]:
def split_data(ratios : list, dataset: torch.utils.data.DataLoader) -> list:
    DATASET_SEED = 12345
    torch_generator = torch.Generator().manual_seed(DATASET_SEED)
    dataset_size = len(dataset)
    sizes = [int(ratio * dataset_size) for ratio in ratios]
    return torch.utils.data.random_split(
        dataset, 
        sizes, 
        generator=torch.Generator().manual_seed(DATASET_SEED))

In [ ]:
'''Test'''
Testbench(split_data)

In [ ]:
train_dataset, val_dataset, test_dataset = split_data([0.7, 0.15, 0.15], dataset)

Написати функцију **getDataLoader** која прима **dataset** и **batch_size** као аргументе и ствара класу **torch.utils.data.DataLoader** са активираним следећим опцијама **shuffle**=True, **num_workers**=0, **drop_last**=True, **pin_memory**=True

In [ ]:
ddef getDataLoader(dataset : torch.utils.data.Dataset , batch_size : int) -> torch.utils.data.DataLoader:
    return torch.utils.data.DataLoader(
        dataset, 
        batch_size=batch_size, 
        shuffle=True, 
        num_workers=0, 
        drop_last=True, 
        pin_memory=True)

In [ ]:
'''Test'''
Testbench(getDataLoader)

Допунити следећу ћелију!

In [ ]:
BATCH_SIZE = 32

train_loader = getDataLoader(train_dataset, BATCH_SIZE)
val_loader = getDataLoader(val_dataset, BATCH_SIZE)

Допунити функцију **get_transfer_learning_model** тако да она враћа већ истренирани модел **resnet50**. Последњи слој мреже треба бити модификован тако да се излаз поклапа са излазом мреже у другој вежби

In [ ]:
from torch import nn

def get_transfer_learning_model():
    model = torchvision.models.resnet50(pretrained=True)
    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, ###)
    return model

In [ ]:
'''Test'''
Testbench(get_transfer_learning_model)

Даље, пребацујемо мрежу на Графичку картицу због бржег тренирања.

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = get_transfer_learning_model()
net.to(device)

Допунити код испод тако да критеријум буде **Cross Entropy Loss**, а да оптимизатор буде **Stohastic Gradient Descent** са **learning rate = 0.001**, a **momentum = 0.9**

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

Допунити функцију **measure_qulity** koja prima модел, сет података, уређај и број BATCH-ева као опциони аргумент. Функција враћа тачност мреже на датом сету података.

In [ ]:
from sklearn.metrics import accuracy_score

def measure_quality(model : nn.Sequential, loader : torch.utils.data.DataLoader, device : any, max_batches: int=None) -> float:
    model.eval()
    iteration_cnt = 0

    all_preds = list()
    all_labels = list()

    with torch.no_grad():
        for i, data in enumerate(loader):
            if max_batches is not None and iteration_cnt == max_batches:
                break

            inputs, labels = data[0].to(device), data[1].to(device)

            outputs = model(inputs)
            _, pred = torch.max(outputs, 1)

            all_preds += list(pred.data.cpu().numpy())
            all_labels += list(labels.data.cpu().numpy())

            iteration_cnt += 1

    model.train()

    return accuracy_score(all_labels, all_preds)

Допунити код којом се тренира модел. Тачност треба да се оцењује на подацима за евалуацију.

In [ ]:
for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # print every 100 mini-batches
            val_accuracy = measure_quality(net, val_loader)
            print(f"epoch {epoch + 1} iter {i + 1} loss: {running_loss / 100} val accuracy: {val_accuracy}")
            running_loss = 0.0

print('Finished Training')